# Codificacion de Valores Faltantes

Asumir que los datos faltantes siempre vendrán en un único formato es un error. Debemos evaluar la data y determinar cómo vienen representados las valores faltantes para luego llevarlos a un formato que pueda ser interpretado por Pandas. 

## Librerias

In [16]:
# Librerias
import pandas as pd
import numpy as np

## Data de Prueba

Creamos un DataFrame a partir de la informacion contenida en un diccionario el cual posee tres variables: x,y,z

In [17]:
df = pd.DataFrame.from_dict(
    dict(
        x = [1, 3, "NA", -99, -98, -99],
        y = ["A", "N/A", "NA", "E", "F", "G"],
        z = [-100, -99, -98, -101, -1, -1]
    )
)

df

,x,y,z
0,1,A,-100
1,3,N/A,-99
2,NA,NA,-98
3,-99,E,-101
4,-98,F,-1
5,-99,G,-1


## Valores Faltantes en la Data

El formato en el que vienen los datos faltantes no es reconocido por Pandas. Esto genera un problema de incosistencia por lo que debemos convertirlos a un formato que sea adecuado para Pandas. En lo que sigue te presentamos varios metodos para visualizar o contar los valores faltantes del DataFrame

### Tabla de booleanos para detectar valores faltantes

In [26]:
df.isna()

,x,y,z
0,False,False,False
1,False,False,False
2,False,False,False
3,False,False,False
4,False,False,False
5,False,False,False


La tabla anterior muestra el valor booleano False para cada dato. Esto indica que el DataFrame posee valores completos lo cual sabemos que no es cierto.  

### Cantidad de valores faltantes por variable

In [27]:
df.isna().sum()

x    0
y    0
z    0
dtype: int64

El resultado muestra que no hay valores faltantes en las variables.

### Cantidad total de valores faltantes en el DataFrame

In [28]:
df.isna().sum().sum()

0

## Valores comúnmente asociados a Valores Faltantes

En cada recoleccion de datos los valores faltantes pueden ser representados de distintas formas. Esto tambien va a depender de qué tipos de datos se están recolectando y de quien los recolecta. Existen algunas representaciones generales que se utilizan en muchos casos y que te presento a continuacion en las siguientes tuplas 

### De tipo texto

In [18]:
common_na_strings = (
    "missing",
    "NA",
    "N A",
    "N/A",
    "#N/A",
    "NA ",
    " NA",
    "N /A",
    "N / A",
    " N / A",
    "N / A ",
    "na",
    "n a",
    "n/a",
    "na ",
    " na",
    "n /a",
    "n / a",
    " a / a",
    "n / a ",
    "NULL",
    "null",
    "",
    "?",
    "*",
    ".",
)

### De tipo numérico

In [19]:
common_na_numbers = (-9, -99, -999, -9999, 9999, 66, 77, 88, -1)

## Detección de Valores Faltantes

### Detectar valores faltantes en columnas mediante el tipo de dato

In [21]:
df.dtypes

x    object
y    object
z     int64
dtype: object

La columna x solo tiene 1 string y Pandas reconoce toda la columna como object lo que sugiere que existen valores faltantes debido a que lo demas valores son de tipo numerico

### Detectar valores faltantes revisando valores unicos en los datos

In [22]:
df.x.unique()

array([1, 3, 'NA', -99, -98], dtype=object)

Como vemos, en la columna x solo hay un elemento con string y los demas son numerico. Sin embargo, Pandas detecta todo el array de tipo object lo que sugiere la presencia de valores faltantes.

Este metodo se aplica a columna por columna y es util solo para datframes pequeños.


### Detectar valores faltantes revisando los valores unicos de las variables de un determinado tipo

In [23]:
(
    df
    .select_dtypes(object)
    .apply(pd.unique)
)

x     [1, 3, NA, -99, -98]
y    [A, N/A, NA, E, F, G]
dtype: object

Lo que se hace es seleccionar las variables de tipo object y regresar sus valores unicos. En este caso vemos que la variable x contiene valores faltantes debido a que Pandas lo detecta como object cuando en su mayoria los datos son de tipo numerico.

### Sustitucion Global de valores asociados a valores faltantes

In [24]:
(
    df
    .replace(
        to_replace=common_na_strings+common_na_numbers,
        value=np.nan
    )
)

,x,y,z
0,1.0,A,-100.0
1,3.0,NaN,NaN
2,NaN,NaN,-98.0
3,NaN,E,-101.0
4,-98.0,F,NaN
5,NaN,G,NaN


Metodo utilizado cuando conoces como estan representados los valores faltantes pero no en que variables se ubican.

### Sustitucion Dirigida de valores asociados a valores faltantes

In [25]:
(
    df
    .replace(
        to_replace={
            'x':{
                -99:np.nan,
                'NA':np.nan
            }
        }
    )
)

,x,y,z
0,1.0,A,-100
1,3.0,N/A,-99
2,NaN,NA,-98
3,NaN,E,-101
4,-98.0,F,-1
5,NaN,G,-1


Este metodo es muy util cuando sabemos en que variables estan ubicados los valores faltantes y como estan representados